In [ ]:
'''
Importing Libraries
'''
import numpy as np
import pandas as pd
import os
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
'''
Loading data
'''
train_df = pd.read_csv('train.csv', encoding = "ISO-8859-1") 
demog_df = pd.read_csv('demog.csv', encoding = "ISO-8859-1") 
submission = pd.read_csv('submission.csv', encoding = "ISO-8859-1")

In [ ]:
train_df = pd.merge(train_df, demog_df, on='HCP_ID', how='left')
#train_df['gender'] = train_df['gender'].fillna('MALE')
train_df.drop('HCP_ID', axis=1, inplace =True)
#train_df = pd.get_dummies(train_df, ['Region', 'Value', 'gender' ])

# Visualizing the distribution of Features of the dataset

In [ ]:
plt.hist(np.sort(train_df['RL'].dropna().values))

In [ ]:
plt.hist(np.sort(submission['RL'].values))

In [ ]:
plt.hist(np.sort(train_df['OLV'].dropna().values))

In [ ]:
plt.hist(np.sort(submission['OLV'].dropna().values))

In [ ]:
plt.hist(np.sort(train_df['RR'].dropna().values))

In [ ]:
plt.hist(np.sort(submission['RR'].dropna().values))

In [ ]:
plt.hist(np.sort(train_df['DRT'].dropna().values))

In [ ]:
plt.hist(np.sort(submission['DRT'].dropna().values))

In [ ]:
plt.hist(np.sort(train_df['DMS'].dropna().values))

In [ ]:
plt.hist(np.sort(submission['DMS'].dropna().values))

In [ ]:
plt.hist(np.sort(train_df['OLA'].dropna().values))

In [ ]:
plt.hist(np.sort(submission['OLA'].dropna().values))

In [ ]:
train_df.head()

In [ ]:
def get_number_of_null_entries(df, var):
    return df[var].shape[0] - df[var].count()

In [ ]:
'''
Number of Null values in every column
'''
for x in train_df.columns:
    print (x, end=" ")
    print (get_number_of_null_entries(train_df, x))

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
def get_error(df, columns):
    rmse = 0
    for x in columns:
        y = df[x].dropna()
        me = y.mean()
        y = df[x].fillna(me)
        rmse = rmse + sqrt(sum((y-me)**2))
    return rmse

In [ ]:
l = ['Speciality_ID', 'Age', 'Region_rural', 'Region_urban', 'Value_H', 'Value_L', 'Value_M', 'Value_U', 'gender_FEMALE', 'gender_MALE']
columns = ['RL', 'P2P', 'OLV', 'RR', 'DRT', 'DMS', 'OLA', 'DEM']
X = train_df[l]

In [ ]:
'''
Formed Clusters using the columns which didnot have Null values.
'''
error_dict = {}
for num_cluster in range(1000, 5000, 100):
    k_means = KMeans(n_clusters= num_cluster, max_iter=150)
    k_means.fit(X)
    labels = k_means.labels_
    error = 0
    for l in range(num_cluster):
        cluster_error = get_error(train_df[columns].iloc[np.where(labels == l)], columns)
        error += cluster_error
    error_dict[num_cluster] = error/num_cluster

In [ ]:
sorted_by_value_keys = sorted(error_dict, key = error_dict.get, reverse = True)
for r in sorted_by_value_keys:
    print (r, error_dict[r])
    break

In [ ]:
'''
The error was decreasing with increase in the number of clusters. I could'not find optimum number of clusters because it was too much time to run.
My final submission is based on clustering with 3000 clusters.
'''
n_clusters = 3000
k_means = KMeans(n_clusters=3000, max_iter=250)
k_means.fit(X)
labels = k_means.labels_
for l in range(n_clusters):
    for x in columns:
        me = train_df[columns].iloc[np.where(labels == l)][x].mean()
        train_df[columns].iloc[np.where(labels == l)][x].replace(-0.1, me)

In [ ]:
train_df['HCP_ID'] = train_df_new['HCP_ID']
train_df = train_df[['HCP_ID','RL', 'P2P', 'OLV', 'RR', 'DRT', 'DMS', 'OLA', 'DEM']]

In [ ]:
train_df.to_csv('shariq_suhail_03021997.csv', index=False)